In [1]:
import numpy as np
import pandas as pd
import math
import random
import pickle
import json

# to make the experimens replicable
random.seed(123456)

In [2]:
df_uffici = pd.read_csv('../data/dg_struttura.csv', dtype=str)
id_label_dict = dict(zip(df_uffici['ID'].values, df_uffici['NOME'].values))

In [3]:
df = pd.read_pickle('../data/atti_processed.pkl')

In [4]:
df.head()

,CODICE_PRATICA,DATA_ATTO,DESCRIZIONE,DESCRIZIONE_TIPO_ATTO,DESCRIZIONE_TIPO_PRATICA,ID_TIPO_PRATICA,LINK_CONTENUTO_BINARIO,OGGETTO,PERSONA,STATUS,UFFICIO_AC,UFFICIO_DG,UFFICIO_SE,publisher
0,20000000011999AD00000005121,2000-01-04,Atto soggetto a pubblicazione per estratto,Decreto soggetto a controllo congiunto,Atto dei dirigenti,MON,http://www.regione.toscana.it/bancadati/atti/C...,DIP. ALESSANDRO ANNUNZIATI (MATR. 14117). ASPE...,004744,2,04246,01931,04516,Acrobat PDFWriter4.0 per Windows
1,20000000021999AD00000005123,2000-01-04,Atto soggetto a pubblicazione per estratto,Decreto soggetto a controllo congiunto,Atto dei dirigenti,MON,http://www.regione.toscana.it/bancadati/atti/C...,DIP. GIOVANNI ORLANDINI (MATR. 14311). ASPETTA...,004744,2,04246,01931,04516,Acrobat PDFWriter4.0 per Windows
2,20000000031999AD00000005018,2000-01-04,Atto soggetto a pubblicazione per estratto,Decreto soggetto a controllo congiunto,Atto dei dirigenti,MON,http://www.regione.toscana.it/bancadati/atti/C...,DIP. LEONELLO TOCCAFONDI. CONGEDO STRAORDINARI...,004744,2,04246,01931,04516,Acrobat PDFWriter4.0 per Windows
3,20000000041999AD00000005116,2000-01-04,Atto soggetto a pubblicazione per estratto,Decreto soggetto a controllo congiunto,Atto dei dirigenti,MON,http://www.regione.toscana.it/bancadati/atti/C...,DIPENDENTI REGIONALI DIVERSI. CONGEDO STRAORDI...,004744,2,04246,01931,04516,Acrobat PDFWriter4.0 per Windows
4,20000000051999AD00000005127,2000-01-04,Atto soggetto a pubblicazione per estratto,Decreto soggetto a controllo congiunto,Atto dei dirigenti,MON,http://www.regione.toscana.it/bancadati/atti/C...,DIP. SILVIA LORENZI (MATR. 13366). CONGEDO STR...,004744,2,04246,01931,04516,Acrobat PDFWriter4.0 per Windows


In [5]:
df.shape

(184318, 14)

### Documents per Office

In [6]:
grouped = df.groupby(['UFFICIO_DG']).count()['CODICE_PRATICA'].sort_values(ascending=False)
grouped

UFFICIO_DG
50073    15748
50004    14375
50117    14059
50115    12691
50000    12439
50116    12275
50003    11145
50005     9865
50202     8858
01937     8794
50006     6788
50119     5999
01934     5882
01943     5186
50125     4892
01928     3886
01946     3855
50007     3592
01025     3257
50002     2893
50201     2205
50123     2160
01931     1748
02090     1566
01027     1528
50127     1445
01923     1141
50112     1102
99999     1060
50124      972
50113      877
50114      677
50122      603
50044      464
50200      186
50026      105
Name: CODICE_PRATICA, dtype: int64

we remove the observations for the following offices

In [7]:
offices = grouped.index[-2:]
offices

Index(['50200', '50026'], dtype='object', name='UFFICIO_DG')

In [8]:
for o in offices:
    df = df[df['UFFICIO_DG'] != o]

In [9]:
df.shape

(184027, 14)

## Dataset Creation

Indipendent Variable:
- 'OGGETTO'
- 'UFFICIO_DG'
- 'DESCRIZIONE_TIPO_ATTO'
- 'DESCRIZIONE_TIPO_PRATICA'
- 'publisher'

In [10]:
df_dataset = df[['OGGETTO', 'UFFICIO_DG']]

In [11]:
df_dataset.shape

(184027, 2)

In [12]:
samples = np.array([x[0] for x in df_dataset.as_matrix()])
labels = np.array([x[1] for x in df_dataset.as_matrix()])

In [13]:
print(samples[10])
print(labels[10])

ELENCO REGIONALE DELLE AZIENDE PRODUTTRICI E FORNITRICI DI PROTESI ED AUSILI DI CUI AL DECRETO N. 1317 DEL 20.03.1998: MODIFICHE ED INTEGRAZIONI
01943


In [14]:
labels = np.array([id_label_dict[x] if x in id_label_dict else 'office_{}'.format(x) for x in labels])

In [15]:
indexes = np.array(range(len(samples)))
np.random.shuffle(indexes)

In [16]:
samples = samples[indexes]
labels = labels[indexes]

### Transform labels to ids

In [17]:
ids_labels = dict(enumerate(sorted(list(set(labels)))))
labels_ids = {v:k for k,v in ids_labels.items()}

In [18]:
labels_ids

{'ALTRI UFFICI': 0,
 'AVVOCATURA REGIONALE                                  ': 1,
 'D.G.  AVVOCATURA                                      ': 2,
 "D.G. COMPETITIVITA' DEL SISTEMA REGIONALE E SVILUPPO D": 3,
 'D.G. PRESIDENZA                                       ': 4,
 'DIPARTIMENTO BILANCIO E FINANZE                       ': 5,
 'DIPARTIMENTO ORGANIZZAZIONE                           ': 6,
 'DIPARTIMENTO ORGANIZZAZIONE E RISORSE                 ': 7,
 'DIPARTIMENTO POLITICHE FORMATIVE E BENI CULTURALI     ': 8,
 'DIPARTIMENTO POLITICHE TERRITORIALI E AMBIENTALI      ': 9,
 'DIPARTIMENTO PRESIDENZA AFFARI LEGISLATIVI E GIURIDICI': 10,
 'DIPARTIMENTO SALUTE E POLITICHE SOLIDARIETA           ': 11,
 'DIPARTIMENTO SVILUPPO ECONOMICO                       ': 12,
 'DIREZIONE AFFARI LEGISLATIVI, GIURIDICI ED ISTITUZIONALI': 13,
 'DIREZIONE AGRICOLTURA E SVILUPPO RURALE': 14,
 "DIREZIONE ATTIVITA' PRODUTTIVE": 15,
 'DIREZIONE CULTURA E RICERCA': 16,
 'DIREZIONE DIFESA DEL SUOLO E PROTEZIONE CIV

In [19]:
with open('../data/dataset/label_index.json', 'w') as f:
    json.dump(labels_ids,f)

In [20]:
encoded_labels = np.array([labels_ids[x] for x in labels])

### Transform the samples into ids

Removed numbers

In [21]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

nltk.download('stopwords')
nltk.download('punkt')

punctuation = ['-', '"', "'", ':', ';', '(', ')', '[', ']', '{', '}', '’', '”', '“', '``', "''"]
stop_words = set(stopwords.words('italian'))
stop_words.update(punctuation)

[nltk_data] Downloading package stopwords to /home/fabio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/fabio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [22]:
def hasnumbers(value):
    return any(c.isdigit() for c in value)

In [23]:
def tokenize_sample(samples, remove_stopwords=True, tokenizer=word_tokenize):
    for sample in samples:
        words = []
        sample = sample.replace('`', ' ')
        sample = sample.replace("'", " ")
        sample = sample.replace("”", ' ')
        sample = sample.replace("“", ' ')
        for w in tokenizer(sample):
            if not hasnumbers(w) and len(w) > 2:
                w = w.replace('_', '')
                if remove_stopwords:
                    if w not in stop_words:
                        words.append(w.lower())
                elif w in stop_words or len(w) > 1:
                    words.append(w.lower())
        yield words

In [24]:
tokenizer = RegexpTokenizer(r'\w+')

In [25]:
samples_tokenized = list(tokenize_sample(samples, remove_stopwords=False, tokenizer=tokenizer.tokenize))

In [26]:
from collections import Counter

counter = Counter()

for sentence in samples_tokenized:
    counter.update(sentence)

In [27]:
counter.most_common()[:20]

[('del', 124592),
 ('per', 106894),
 ('della', 69735),
 ('dell', 43670),
 ('regionale', 38600),
 ('art', 35297),
 ('delle', 31919),
 ('dei', 29820),
 ('impegno', 29750),
 ('comune', 29678),
 ('alla', 27894),
 ('approvazione', 27840),
 ('toscana', 27650),
 ('liquidazione', 24508),
 ('all', 22846),
 ('con', 22583),
 ('autorizzazione', 20523),
 ('progetto', 19461),
 ('spesa', 18459),
 ('sensi', 18230)]

The words are enumerated in ascending order so that we can select the top k words during the classification task

In [28]:
words = [w for w,_ in counter.most_common()]

In [31]:
len(words) + 3

52396

In [32]:
id_word_dict = dict(enumerate([w for w in words],3))
word_id_dict = {v:k for k,v in id_word_dict.items()}

In [33]:
with open('../data/dataset/id_word_dict.json','w') as f:
    json.dump(id_word_dict,f)

In [34]:
pad_char = 0
start_char=1
oov_char=2

In [35]:
def to_sequece_of_ids(tokenized_samples, word_id_dict):
    results = []
    for sample in tokenized_samples:
        encoded_sample = []
        for w in sample:
            if w in word_id_dict:
                encoded_sample.append(word_id_dict[w])
            else:
                encoded_sample.append(oov_char)
        results.append(encoded_sample)
    return results

In [36]:
encoded_samples = np.array(to_sequece_of_ids(samples_tokenized, word_id_dict))

In [37]:
encoded_samples[:4]

array([list([837, 287, 4, 68, 5, 373, 2079, 476, 703, 8655, 18, 907, 2849, 4875, 2880, 609, 954, 355, 4, 31, 502, 826, 189, 4382, 123, 2020, 10269, 10270, 10, 504, 468, 682, 1062, 1287, 569, 1639, 1368, 14, 106, 165, 53, 21, 4, 27]),
       list([68, 22, 6, 8, 43, 455, 3, 56, 13, 80, 955, 6358, 1266, 5, 373, 1844, 98, 179, 955, 14, 106, 165, 53]),
       list([1038, 194, 3136, 15236, 62, 291, 1534, 2326, 18]),
       list([706, 6503, 5, 332, 957, 3, 19, 228, 644, 7])], dtype=object)

In [38]:
max(np.max(encoded_samples))

52374

## Create Train and Test Set

- shuffle data
- split the dataset into 80-20

In [39]:
from sklearn.model_selection import StratifiedShuffleSplit

In [40]:
split_train_test = StratifiedShuffleSplit(1,test_size=0.2, random_state=123456)

for train, test in split_train_test.split(encoded_samples, encoded_labels):
    train_samples, test_samples = encoded_samples[train], encoded_samples[test]
    train_labels, test_labels = encoded_labels[train], encoded_labels[test]

## Save the Dataset

In [41]:
np.savez_compressed('../data/dataset/atti_dataset.npz',
                   x_train=train_samples, y_train=train_labels,
                   x_test=test_samples, y_test=test_labels
                   )